# ANALIZA PODATKOV O HOTELIH

V tem delu bomo pridobljene podatke predelali, analizirali in jih primerjali.

In [2]:
# tabele bomo analizirali s pomočjo pandas, zato ga naložimo
import pandas as pd

# ker so razpredelnice velike, povemo, naj vedno izpiše le prvih 10 vrstic
pd.set_option('display.max_rows', 10)

# naložimo csv-je
amsterdam = pd.read_csv('amsterdam.csv')
split = pd.read_csv('split.csv')
lizbona = pd.read_csv('lizbona.csv') 

Kot pričakovano, nas najprej zanima katera lokacija ima v povprečju najdražje oz. najcenejše prenočitve in katera v poprečju najvišje oz. najnižje ocene.